In [ ]:
#
# Tests the ability to use springs layout selectively on subgraphs
# ... so, if you keep the placement of the nodes from an initial all-nodes layout
# ... then you can update the placement of a subset of the nodes
#
# However... because that's not the only transform done to make this all work
# (think about the treemap component placement), then there's always a scaling factor
# involved... and that breaks the ability to apply the spring layout to a subset of 
# nodes...
#

import polars   as pl
import networkx as nx
import random
import copy
import rtsvg
import linknode_graph_patterns
from rtsvg.polars_spring_layout import PolarsSpringLayout
rt   = rtsvg.RACETrack()
'''
g    = linknode_graph_patterns.LinkNodeGraphPatterns().__pattern_mesh__()
_lu_ = {'fm':[], 'to':[]}
for n in g.nodes():
    for nbor in g.neighbors(n): _lu_['fm'].append(n), _lu_['to'].append(nbor)
df        = pl.DataFrame(_lu_)
_to_fix_  = {'node_0_5', 'node_7_4', 'node_8_6', 'node_5_8'}
'''

_base_network_ = '0'
_base_dir_     = '../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df        = pl.DataFrame(_lu_)
_relates_ = [('fm','to')]
g         = rt.createNetworkXGraph(df, _relates_)
_to_fix_  = {'110', '193', '201', '245', '259', '264', '61', '8', '91'}

_relates_ = [('fm','to')]
_colors_  = {n:'red' for n in _to_fix_}
pos       = PolarsSpringLayout(g).results()
new_pos   = copy.deepcopy(pos)
for n in _to_fix_: new_pos[n] = (new_pos[n][0] + 100*(random.random()-0.5), new_pos[n][1] + 100*(random.random()-0.5))
new_new_pos = copy.deepcopy(new_pos)
new_new_pos = PolarsSpringLayout(g, pos=new_new_pos, static_nodes=set(g.nodes()) - _to_fix_).results()
params      = {'w':384, 'h':384, 'node_color':_colors_}
rt.tile([rt.link(df, _relates_, pos,         **params),
         rt.link(df, _relates_, new_pos,     **params),
         rt.link(df, _relates_, new_new_pos, **params)], spacer=10)

In [ ]:
#
# So... maybe it's possible to rescale the graph based on the expected distances
# ... in this case (mesh graph), all edges should be 1.0 ... so... how to take
# ... a random set of positions & then rescale them?
#
_d_sum_, _d_samples_, _rms_sum_ = 0.0, 0, 0.0
for n in g.nodes():
    for nbor in g.neighbors(n):
        _w_                  =  g[n][nbor]['weight']
        _xy_n_, _xy_nbor_    =  pos[n], pos[nbor]
        _d_                  =  rt.segmentLength((_xy_n_, _xy_nbor_))
        _w_diff_             =  _w_ - _d_
        _rms_sum_            += _w_diff_**2
        _d_sum_, _d_samples_ = _d_sum_ + _d_, _d_samples_ + 1
print(_d_sum_, _d_samples_, _d_sum_ / _d_samples_) # 359.24853900508407 288 1.2473907604343197 -- example from the mesh graph

In [ ]:
#
# Grid Search Version of This Optimization
#
def rms(pos, g, x_scale, y_scale):
    _d_sum_, _d_samples_, _rms_sum_ = 0.0, 0, 0.0
    for n in g.nodes():
        for nbor in g.neighbors(n):
            _w_                  =  g[n][nbor]['weight']
            _xy_n_, _xy_nbor_    =  (pos[n][0]*x_scale, pos[n][1]*y_scale), (pos[nbor][0]*x_scale, pos[nbor][1]*y_scale)
            _d_                  =  rt.segmentLength((_xy_n_, _xy_nbor_))
            _w_diff_             =  _w_ - _d_
            _rms_sum_            += _w_diff_**2
            _d_sum_, _d_samples_ = _d_sum_ + _d_, _d_samples_ + 1
    _rms_ = _rms_sum_ / _d_samples_
    _rms_ = _rms_**0.5
    return _rms_

_lu_ = {'x_scale':[], 'y_scale':[], 'rms':[]}
_inc_     = 0.05
_x_scale_ = 0.5
while _x_scale_ < 1.5:
    _y_scale_ = 0.5
    while _y_scale_ < 1.5:
        _rms_ = rms(pos, g, _x_scale_, _y_scale_)
        _lu_['x_scale'].append(_x_scale_)
        _lu_['y_scale'].append(_y_scale_)
        _lu_['rms'].append(_rms_)
        _y_scale_ += _inc_
    _x_scale_ += _inc_

rt.tile([rt.xy(pl.DataFrame(_lu_), x_field='x_scale', y_field='rms', line_groupby_field='y_scale', color_by='y_scale', dot_size=None),
         rt.xy(pl.DataFrame(_lu_), x_field='y_scale', y_field='rms', line_groupby_field='x_scale', color_by='x_scale', dot_size=None)], spacer=10)

In [ ]:
from math import sqrt
_lu_ = {'n':[], 'nbor':[], 'w':[], 'n_x':[], 'nbor_x':[], 'n_y':[], 'nbor_y':[]}
for n in g.nodes():
    for nbor in g.neighbors(n):
        _w_                  =  g[n][nbor]['weight']
        _xy_n_, _xy_nbor_    =  pos[n], pos[nbor]
        _lu_['n'].append(n),                 _lu_['nbor'].append(nbor),             _lu_['w'].append(_w_)
        _lu_['n_x'].append(_xy_n_[0]),       _lu_['n_y'].append(_xy_n_[1])
        _lu_['nbor_x'].append(_xy_nbor_[0]), _lu_['nbor_y'].append(_xy_nbor_[1])
df_for_rms = pl.DataFrame(_lu_)
def rmsWithPolars(x_scale, y_scale):
    __n_x__, __n_y__       = pl.col('n_x'), pl.col('n_y')
    __nbor_x__, __nbor_y__ = pl.col('nbor_x'), pl.col('nbor_y')
    _df_ = df_for_rms.with_columns(((__n_x__*x_scale - __nbor_x__*x_scale)**2 + (__n_y__*y_scale - __nbor_y__*y_scale)**2).alias('x2'))
    _df_ = _df_.with_columns((pl.col('x2')**0.5).alias('d'))
    _df_ = _df_.with_columns(((df_for_rms['w'] - _df_['d'])**2).alias('rms'))
    return sqrt(_df_['rms'].sum()/len(_df_))
rmsWithPolars(1.1, 0.9), rms(pos, g, 1.1, 0.9) # compares the two approaches

In [ ]:
#%%timeit
#rmsWithPolars(1.0, 1.0) # 255 μs ± 5.64 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

In [ ]:
#%%timeit
#rms(pos, g, 1.0, 1.0) # 3.18 ms ± 74.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [ ]:
rmsWithPolars(1.0, 0.1), rmsWithPolars(1.0, 1.0), rmsWithPolars(1.0, 10.0), rmsWithPolars(1.0, 100.0)

In [ ]:
rmsWithPolars(0.1, 1.0), rmsWithPolars(1.0, 1.0), rmsWithPolars(10.0, 1.0), rmsWithPolars(100.0, 1.0)

In [ ]:
#
# From Google AI
#
import numpy as np

def calculate_parabola_coefficients(p1, p2, p3):
    """
    Calculates the coefficients (a, b, c) of a parabola y = ax^2 + bx + c
    that passes through three given points.

    Args:
        p1 (tuple): The first point (x1, y1).
        p2 (tuple): The second point (x2, y2).
        p3 (tuple): The third point (x3, y3).

    Returns:
        tuple: A tuple containing the coefficients (a, b, c).
               Returns None if the points are collinear or cannot form a unique parabola.
    """
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3

    # Construct the matrix A and vector B for the system of equations
    # A * [a, b, c]^T = B
    A = np.array([
        [x1**2, x1, 1],
        [x2**2, x2, 1],
        [x3**2, x3, 1]
    ])

    B = np.array([y1, y2, y3])

    try:
        # Solve the system of equations for a, b, c
        coefficients = np.linalg.solve(A, B)
        return tuple(coefficients)
    except np.linalg.LinAlgError:
        # This error occurs if the matrix A is singular,
        # which happens when the three points are collinear.
        print("Error: The three points are collinear or cannot form a unique parabola.")
        return None

def get_parabola_equation(a, b, c):
    """
    Returns a string representation of the parabola's equation.
    """
    if a is None:
        return "No unique parabola found."
    return f"y = {a:.4f}x^2 + {b:.4f}x + {c:.4f}"

# Example usage:
point1 = (1, 2)
point2 = (2, 1)
point3 = (3, 4)

a, b, c = calculate_parabola_coefficients(point1, point2, point3)

if a is not None:
    print(f"Parabola coefficients: a={a}, b={b}, c={c}")
    print(f"Parabola equation: {get_parabola_equation(a, b, c)}")

# Example with collinear points:
collinear_point1 = (0, 0)
collinear_point2 = (1, 1)
collinear_point3 = (2, 2)

a_collinear, b_collinear, c_collinear = calculate_parabola_coefficients(
    collinear_point1, collinear_point2, collinear_point3
)

In [ ]:
rmsWithPolars(1.0, 0.1), rmsWithPolars(1.0, 1.0), rmsWithPolars(1.0, 10.0), rmsWithPolars(1.0, 100.0)
# (0.6143791627370341, 0.49946486972850823, 5.964854550221141, 66.29257850111811)

In [ ]:
# Taking the last cells result and organizing them into (x, y) format
p0 = (0.1,    0.6143791627370341)
p1 = (1.0,    0.49946486972850823)
p2 = (10.0,   5.964854550221141)
p3 = (100.0, 66.29257850111811)

# Now, taking the three points that form the dip f(x0) > f(x1) < f(x2)
# ... which are p0, p1, and p2 from above
calculate_parabola_coefficients(p0, p1, p2) 

In [ ]:
# Which yields the following
_a_, _b_, _c_ = 0.07423717856989777, -0.20934344421413842, 0.6345711353727489

import math

#
# From Google AI -- "solve quadratic equation in python"
#
def solve_quadratic_equation(a, b, c):
    """
    Solves a quadratic equation of the form ax^2 + bx + c = 0.

    Args:
        a (float): Coefficient of x^2.
        b (float): Coefficient of x.
        c (float): Constant term.

    Returns:
        tuple: A tuple containing the solutions (sol1, sol2).
               If a=0, it indicates an invalid input.
    """
    if a == 0:
        print("Error: 'a' cannot be zero for a quadratic equation.")
        return None, None

    # Calculate the discriminant
    discriminant = (b**2) - (4 * a * c)
    if discriminant < 0:
        print("No real solutions exist.")
        return None, None

    # Find the two solutions
    sol1 = (-b - math.sqrt(discriminant)) / (2 * a)
    sol2 = (-b + math.sqrt(discriminant)) / (2 * a)

    return sol1, sol2

solve_quadratic_equation(_a_, _b_, _c_)